## Multilabel Classification of Frames of Futurama

Dataset Source: https://www.kaggle.com/datasets/gonzalorecioc/futurama-frames-with-characteronscreen-data

#### Import Necessary Libraries

In [1]:
import os, sys

import numpy as np
import pandas as pd

import PIL.Image
from PIL import ImageDraw, ImageFont

import transformers
from transformers import AutoTokenizer, ViTForImageClassification
from transformers import TrainingArguments, Trainer, set_seed, ViTImageProcessor

import datasets
from datasets import Dataset, DatasetDict, load_dataset

import torch

import evaluate

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

!git lfs install

NOTE: Redirects are currently not supported in Windows or MacOs.


Git LFS initialized.


##### Display Version of Relevant Libraries

In [2]:
print("Python:".rjust(15), sys.version[0:6])
print("Pandas:".rjust(15), pd.__version__)
print("NumPy:".rjust(15), np.__version__)
print("Torch:".rjust(15), torch.__version__)
print("Transformers:".rjust(15), transformers.__version__)
print("Datasets:".rjust(15), datasets.__version__)
print("Evaluate:".rjust(15), evaluate.__version__)

        Python: 3.9.12
        Pandas: 1.5.0
         NumPy: 1.23.3
         Torch: 1.12.1
  Transformers: 4.26.1
      Datasets: 2.8.0
      Evaluate: 0.2.2


#### Make Necessary Updates to Metadata File

In [3]:
parent_dir = "/Users/briandunn/Desktop/Vit_Image_Datasets/ML CLF/Futurama Frames"

df = pd.read_csv(os.path.join(parent_dir, "data.csv"))

df = df.rename(columns={"file" : "file_name"})

df.to_csv(os.path.join(parent_dir, "img", "metadata.csv"), index=False)

df

,file_name,isLeela,isFry,isBender
0,Futurama_1_04081.png,0,1,0
1,Futurama_2_17161.png,1,1,0
2,Futurama_7_14497.png,1,1,1
3,Futurama_6_23881.png,0,0,0
4,Futurama_5_02065.png,1,0,0
...,...,...,...,...
9151,Futurama_2_06193.png,0,1,0
9152,Futurama_1_10801.png,0,0,0
9153,Futurama_6_17593.png,1,1,1
9154,Futurama_6_00313.png,0,1,0


#### Ingest Dataset

In [4]:
dataset = load_dataset(os.path.join(parent_dir, "img"), split='train')
dataset
dataset[12]

A Jupyter Widget

Using custom data configuration img-00d4e2cc6bbffb36


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Dataset imagefolder downloaded and prepared to /Users/briandunn/.cache/huggingface/datasets/imagefolder/img-00d4e2cc6bbffb36/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=180x135>,
 'isLeela': 0,
 'isFry': 0,
 'isBender': 0}

#### Split Dataset into Training & Testing Datasets

In [5]:
train_test_ds = dataset.train_test_split(test_size=0.20)

ds = DatasetDict({
    'train': train_test_ds['train'], 
    'eval': train_test_ds['test']
    })

print('Training data shape:', ds['train'].shape)
print('Validation data shape:', ds['eval'].shape)

Training data shape: (7324, 4)
Validation data shape: (1832, 4)


#### Define List of Labels & Label Conversion Dictionaries

In [6]:
labels = [x for x in df.columns if x not in ['image', 'file_name']]

num_labels=len(labels)
id2label={str(i): c for i, c in enumerate(labels)}
label2id={c: str(i) for i, c in enumerate(labels)}

#### Basic Values/Constants

In [7]:
set_seed = 42
NUM_OF_EPOCHS = 8

BATCH_SIZE = 8
LEARNING_RATE = 2e-5

REPORTS_TO = "tensorboard"
STRATEGY = "epoch"

MODEL_CKPT = "google/vit-base-patch16-224"
MODEL_NAME = MODEL_CKPT.split("/")[-1] + "-Futurama_Image_multilabel_clf"

METRIC_NAME = "f1"
DEVICE = torch.device("mps")

#### Load ViT Feature Extractor

In [8]:
feature_extractor = ViTImageProcessor.from_pretrained(MODEL_CKPT)

#### Preprocess Dataset

In [9]:
def transform(sample_batch):
    # Convert list of PIL Images into pixel values
    inputs = feature_extractor([x for x in sample_batch['image']], return_tensors="pt")
    
    # Prep Labels
    labels_batch = {k: sample_batch[k] for k in sample_batch.keys() if k in labels}
    
    labels_matrix = np.zeros((len(sample_batch['image']), len(labels)))
    
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    
    inputs["labels"] = labels_matrix.tolist()
    
    return inputs

#### Apply Transform Function to Dataset

In [10]:
print(ds['train'][12])

prepped_ds = ds.with_transform(transform)
prepped_ds
prepped_ds['train'][12]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=180x135 at 0x7FE688150F40>, 'isLeela': 0, 'isFry': 0, 'isBender': 0}


{'pixel_values': tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 'labels': [0.0, 0.0, 0.0]}

#### Instantiate Model

In [11]:
model = ViTForImageClassification.from_pretrained(
    MODEL_CKPT, 
    problem_type="multi_label_classification",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(DEVICE)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Define Data Collator Function

In [12]:
def data_collator(batch):
    return {
        'pixel_values' : torch.stack([x['pixel_values'] for x in batch]),
        'labels' : torch.tensor([x['labels'] for x in batch]),
    }

#### Define Function to Compute Metrics 

In [13]:
def multi_label_metrics(predictions, 
                        labels, 
                        threshold=0.5):
    '''
    This function calculates & returns metrics 
    for a multilabel classification analysis.
    '''
    
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    preds = preds[0] if isinstance(preds, 
                    tuple) else preds
    results = multi_label_metrics(
                    predictions=preds, 
                    labels=labels)
    return results

#### Define Training Arguments

In [14]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    remove_unused_columns=False,
    num_train_epochs=NUM_OF_EPOCHS,
    evaluation_strategy=STRATEGY,
    save_strategy=STRATEGY,
    learning_rate=LEARNING_RATE,
    disable_tqdm=False,
    report_to=REPORTS_TO,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    logging_first_step=True,
    hub_private_repo=True,
    push_to_hub=True
)

#### Instantiate Trainer

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=prepped_ds['train'],
    eval_dataset=prepped_ds['eval'],
    tokenizer=feature_extractor
)

Cloning https://huggingface.co/DunnBC22/vit-base-patch16-224-Futurama_Image_multilabel_clf into local empty directory.


#### Train Model

In [16]:
train_results = trainer.train()

/Users/briandunn/Documents/nlpnn/nlp_ml/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7324
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7328
  Number of trainable parameters = 85800963


A Jupyter Widget

{'loss': 0.6884, 'learning_rate': 1.999727074235808e-05, 'epoch': 0.0}
{'loss': 0.2456, 'learning_rate': 1.8635371179039304e-05, 'epoch': 0.55}


***** Running Evaluation *****
  Num examples = 1832
  Batch size = 8


A Jupyter Widget

...

A Jupyter Widget

Saving model checkpoint to vit-base-patch16-224-Futurama_Image_multilabel_clf/checkpoint-7328
Configuration saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/checkpoint-7328/config.json
Model weights saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/checkpoint-7328/pytorch_model.bin
Image processor saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/checkpoint-7328/preprocessor_config.json


{'eval_loss': 0.06680033355951309, 'eval_f1': 0.981235926945209, 'eval_roc_auc': 0.9835841216729171, 'eval_accuracy': 0.9667030567685589, 'eval_runtime': 1637.4794, 'eval_samples_per_second': 1.119, 'eval_steps_per_second': 0.14, 'epoch': 8.0}


Image processor saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from vit-base-patch16-224-Futurama_Image_multilabel_clf/checkpoint-4580 (score: 0.98175456135966).


{'train_runtime': 171163.2402, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.043, 'train_loss': 0.027292877175401912, 'epoch': 8.0}


#### Save & Log Model

In [17]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Saving model checkpoint to vit-base-patch16-224-Futurama_Image_multilabel_clf
Configuration saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/config.json
Model weights saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/pytorch_model.bin
Image processor saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/preprocessor_config.json
Saving model checkpoint to vit-base-patch16-224-Futurama_Image_multilabel_clf
Configuration saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/config.json
Model weights saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/pytorch_model.bin
Image processor saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


A Jupyter Widget

A Jupyter Widget

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/DunnBC22/vit-base-patch16-224-Futurama_Image_multilabel_clf
   699db9b..46f164c  main -> main

To https://huggingface.co/DunnBC22/vit-base-patch16-224-Futurama_Image_multilabel_clf
   46f164c..0d54221  main -> main



***** train metrics *****
  epoch                    =                8.0
  train_loss               =             0.0273
  train_runtime            = 1 day, 23:32:43.24
  train_samples_per_second =              0.342
  train_steps_per_second   =              0.043


#### Evaluate, Log, & Save Metrics

In [18]:
metrics = trainer.evaluate(prepped_ds['eval'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 1832
  Batch size = 8


A Jupyter Widget

***** eval metrics *****
  epoch                   =        8.0
  eval_accuracy           =     0.9672
  eval_f1                 =     0.9818
  eval_loss               =     0.0592
  eval_roc_auc            =     0.9842
  eval_runtime            = 0:27:32.60
  eval_samples_per_second =      1.109
  eval_steps_per_second   =      0.139


#### Push Model to Hub (My Profile!!!)

In [19]:
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "tags": ['image-classification'],
}

if args.push_to_hub:
    trainer.push_to_hub('All DUNN!!!', **kwargs)
else:
    trainer.create_model_card(**kwargs)

Saving model checkpoint to vit-base-patch16-224-Futurama_Image_multilabel_clf
Configuration saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/config.json
Model weights saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/pytorch_model.bin
Image processor saved in vit-base-patch16-224-Futurama_Image_multilabel_clf/preprocessor_config.json


A Jupyter Widget

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/DunnBC22/vit-base-patch16-224-Futurama_Image_multilabel_clf
   0d54221..5606d1d  main -> main

To https://huggingface.co/DunnBC22/vit-base-patch16-224-Futurama_Image_multilabel_clf
   5606d1d..b08985e  main -> main



### Notes & Other Takeaways From This Project
****
- I am pleasantly surprised at how quickly this model was able to find the optimal solution.
****

### Citations
- Model Checkpoint
    - @misc{wu2020visual,
      title={Visual Transformers: Token-based Image Representation and Processing for Computer Vision}, 
      author={Bichen Wu and Chenfeng Xu and Xiaoliang Dai and Alvin Wan and Peizhao Zhang and Zhicheng Yan and Masayoshi Tomizuka and Joseph Gonzalez and Kurt Keutzer and Peter Vajda},
      year={2020},
      eprint={2006.03677},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}

    - @inproceedings{deng2009imagenet,
      title={Imagenet: A large-scale hierarchical image database},
      author={Deng, Jia and Dong, Wei and Socher, Richard and Li, Li-Jia and Li, Kai and Fei-Fei, Li},
      booktitle={2009 IEEE conference on computer vision and pattern recognition},
      pages={248--255},
      year={2009},
      organization={Ieee}
}